In [1]:
import redis
import pymongo
import json
import pandas as pd
from bson.json_util import dumps

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
MAX_EXPIRE_DURATION = 24 * 3600 

In [3]:
def configure(max_redis_connections = 2): 
    client = pymongo.MongoClient()
    globals()['client'] = client
    cache_pool = redis.ConnectionPool(host = 'localhost', port = 6379, db = 0, max_connections = max_redis_connections)
    globals()['cache_pool'] = cache_pool

In [4]:
configure()

In [5]:
def get_database_connection():  
    '''
    This method should be called to get a connection to the MongoDB server
    '''
    return globals()['client']

In [6]:
def get_cache_connection():  
    '''
    This method should be called to get a connection to the Redis server  
    '''
    return redis.StrictRedis(connection_pool = globals()['cache_pool'])

In [36]:
def search(query):  
    '''
    The function perform the query search. 
    It first checks the cache to see whether the key exists. 
    If yes, then the query result corresponding to cache are retrieved from Redis, 
    else an query is fired to database.
    '''
    cache = get_cache_connection()
    if cache.exists(query):
        print('Exist in Cache')
        results = lookup_cache_key(query)
        return(results)
    else:
        print('Do not exist in Cache')
        results = database_lookup(query)
        return(results)

In [27]:
def lookup_cache_key(cache_key):  
    '''
    Lookup a key in Redis cache
    '''
    cache = get_cache_connection()
    ids = cache.get(cache_key)
    return ids

In [30]:
def database_lookup(query):  
    '''
    This will run when cache is not available
    This function will search the results in MongoDB
    and saved the cache to the Redis
    '''
    myquery = {"content":{"$regex":query,"$options" :'i'}}
    client = get_database_connection()
    db = client["tweet_database"]
    tweets = db.tweets_collection
    twts = list(tweets.find(myquery))
    save_query_to_cache(query, twts)
    return twts

In [35]:
def save_query_to_cache(cache_key, twts):  
    '''
    Save the query result of into Redis cache
    Also, given the expiry date for the duration
    '''
    cache = get_cache_connection()
    serializedObj = dumps(twts)
    cache.set(cache_key, serializedObj)
    cache.expire(cache_key, MAX_EXPIRE_DURATION)

## Check the performance for searching a keyword with cache and without cache

In [38]:
## Run the query first time
import time
start_time = time.time()
search("covid19")
print("--- %s seconds ---" % (time.time() - start_time))

Do not exist in Cache
--- 0.22595620155334473 seconds ---


In [39]:
## Run the same query again, change the time difference
import time
start_time = time.time()
search("covid19")
print("--- %s seconds ---" % (time.time() - start_time))

Exist in Cache
--- 0.003137350082397461 seconds ---


In [37]:
## For now we have delete the query from database
redis.StrictRedis(connection_pool = globals()['cache_pool']).delete('covid19')


1

## Check the performance for searching a Hashtag with cache and without cache

In [20]:
def database_lookup(query):  
    '''
    This will run when cache is not available
    This function will search the results in MongoDB
    and saved the cache to the Redis
    '''
    myquery = {"content":{"$regex":query,"$options" :'i'}}
    client = get_database_connection()
    db = client["tweet_database"]
    tweets = db.tweets_collection
    twts = list(tweets.find(myquery))
    save_query_to_cache(query, twts)
    return twts

In [21]:
## Run the query first time
import time
start_time = time.time()
search("#trump")
print("--- %s seconds ---" % (time.time() - start_time))

Do not exist in Cache
--- 0.1576838493347168 seconds ---


In [22]:
## Run the query first time
import time
start_time = time.time()
search("#trump")
print("--- %s seconds ---" % (time.time() - start_time))

Exist in Cache
--- 0.003531932830810547 seconds ---


## Check the performance for searching a username with cache and without cache

In [24]:
def database_lookup(query):  
    '''
    This will run when cache is not available
    This function will search the results in MongoDB
    and saved the cache to the Redis
    '''
    myquery = {"user_name":query}
    client = get_database_connection()
    db = client["tweet_database"]
    tweets = db.tweets_collection
    twts = list(tweets.find(myquery))
    save_query_to_cache(query, twts)
    return twts

In [25]:
## Run the query first time
import time
start_time = time.time()
search("Howard")
print("--- %s seconds ---" % (time.time() - start_time))

Do not exist in Cache
--- 0.021148681640625 seconds ---


In [26]:
## Run the query second time
import time
start_time = time.time()
search("Howard")
print("--- %s seconds ---" % (time.time() - start_time))

Exist in Cache
--- 0.0027000904083251953 seconds ---
